# qiskit_alt

This python package provides a thin wrapper around some features of Qiskit that have been (re-)implemented in Julia and provides a Python interface. The input and output of the Python interface are the same as the input and output to Python qiskit. At present, we have prepared two high level demonstrations

* Performing the Jordan-Wigner transform from a Fermionic operator to a Pauli operator.

* Computing the Fermionic operator from integrals computed by `pyscf`.

In both cases, we will see that the `qiskit_alt` implementation is much more performant.

We have also prepared some lower-level demonstrations of performance gains

* Converting an operator from the computational basis to the Pauli basis.

* Creating a `SparsePauliOp` from a list of strings

The Python package has been installed in a virtual environment created with `python -m venv ./env`. The Julia packages have been installed in a local environment in the standard way, via a spec in `Project.toml` file.

When we import the package `qiskit_alt`, the Julia environment is also activated.

In [1]:
%time import qiskit_alt  # The message below is from Julia

┌ Warning: Package ElectronicStructure does not have PyCall in its dependencies:
│ - If you have ElectronicStructure checked out for development and have
│   added PyCall as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ElectronicStructure
└ Loading PyCall into ElectronicStructure from project dependency, future warnings for ElectronicStructure are suppressed.


CPU times: user 1.54 s, sys: 603 ms, total: 2.15 s
Wall time: 1.8 s


  Activating project at `~/myrepos/quantum_repos/qiskit_alt`


We assume that no one is familiar with Julia, much less with `pyjulia`, the package we use to call Julia from Python. So, we inject a bit of tutorial.

The default `Module` in Julia `Main` is available. You can think of it as a namespace. And, as always, objects from the `Module` `Base` have been imported into `Main`.

As an example of how `pyjulia` works, we create an `Array` of `Float64` zeros on the Julia side, and they are automatically copied to a numpy array when returned to Python.

In [2]:
qiskit_alt.Main.zeros(10)

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall ~/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

There are several ways to call Julia from Python and vice versa, and to specifiy features such as the copying vs. sharing semantics. We won't go into any of this in this demo.

## Electronic structure

Part of a workflow for, say, VQE involves using qiskit-nature to do the following:
* Get a description of a model Hamiltonian from the package `pyscf` by passing it a description of the geometry of a molecule.
* Convert that description of a Hamiltonian to a qiskit-native Fermionic operator.
* Convert the Fermionic operator to a qubit operator expressed in the Pauli basis.

The last step above may be done in several ways, one of which is known as the Jordan-Wigner transform. It is this step that we will benchmark here.

### qiskit-nature

First, we see how this is done in qiskit-nature. We need to specify the geometry of the molecule and the
[basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)). We choose `sto3g`, one of the smallest, simplest, basis sets.

In [3]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# Specify the geometry of the H_2 molecule
geometry = [['H', [0., 0., 0.]],
            ['H', [0., 0., 0.735]]]

basis = 'sto3g'

Then, we compute the fermionic Hamiltonian like this.

In [4]:
molecule = Molecule(geometry=geometry,
                     charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF)

from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

fermionic_hamiltonian = second_q_op[0]

The Jordan-Wigner transform is performed like this.

In [5]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
nature_qubit_op = qubit_converter.convert(fermionic_hamiltonian)
nature_qubit_op.primitive

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.81054798+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### qiskit_alt

The only high-level code in `qiskit_alt` was written to support this demo. So doing the JW-transform is less verbose.

In [6]:
fermi_op = qiskit_alt.fermionic_hamiltonian(geometry, basis)
pauli_op = qiskit_alt.jordan_wigner(fermi_op)
pauli_op.simplify() # The Julia Pauli operators use a different sorting convention; we sort again for comparison. 

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.09057899+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

Note that the constant term differs. The qiskit-nature version ignores the nuclear-repulsion term. I need to open an issue about whether and how to handle it.

### Benchmarking

Computing the Hamiltonian for a larger molecule or a larger basis set takes more time and produces a Hamiltonian with more factors and terms. Here we compare the performance of `qiskit_alt` and `qiskit-nature` on combinations of $\text{H}_2$ and $\text{H}_2\text{O}$ molecules for several basis sets.

First we benchmark qiskit-nature, and record the times in `nature_times`.

In [7]:
%run ../bench/jordan_wigner_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 7.77 ms
geometry=h2_geometry, basis='631g' 107.39 ms
geometry=h2_geometry, basis='631++g' 568.65 ms
geometry=h2o_geometry, basis='sto3g' 670.76 ms
geometry=h2o_geometry, basis='631g' 19033.59 ms


[7.766760280355811,
 107.39048114046454,
 568.6548941768706,
 670.7601774483919,
 19033.586104866117]

Next we benchmark qiskit_alt, and record the times in `alt_times`.

In [8]:
%run ../bench/jordan_wigner_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 0.88 ms
geometry=h2_geometry, basis='631g' 4.16 ms
geometry=h2_geometry, basis='631++g' 21.18 ms
geometry=h2o_geometry, basis='sto3g' 21.88 ms
geometry=h2o_geometry, basis='631g' 587.96 ms


[0.8847979828715324,
 4.156961990520358,
 21.18351231329143,
 21.878422936424613,
 587.9554405808449]

We compare the relative performance.

In [9]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[8.778004053704427,
 25.83388575247032,
 26.844221381554014,
 30.658525040745374,
 32.37249762679757]

We see that
* qiskit_alt is at least ten times faster
* The relative performance increases as the problem in some sense gets larger.

In fact, another problem, not shown here, finishes in 18s with qiskit_alt and in 5730s in qiskit-nature.
In this case, `qiskit_alt` is 320 times faster than `qiskit-nature`. I don't have an idea about the origin of this scaling.

### Computing the Fermonic operator

Computing the Fermionic operator from the output of `pyscf` is also much more efficient in `qiskit_alt`.

We benchmark qiskit-nature computing the fermionic Hamiltonian

In [10]:
%run ../bench/fermionic_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 58.56 ms
geometry=h2_geometry, basis='631g' 100.33 ms
geometry=h2_geometry, basis='631++g' 293.21 ms
geometry=h2o_geometry, basis='sto3g' 451.87 ms
geometry=h2o_geometry, basis='631g' 26101.98 ms


[58.56378711760044,
 100.32746009528637,
 293.211798556149,
 451.8730656243861,
 26101.98276070878]

We benchmark qiskit_alt computing the fermionic Hamiltonian.

In [11]:
%run ../bench/fermionic_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 57.63 ms
geometry=h2_geometry, basis='631g' 60.60 ms
geometry=h2_geometry, basis='631++g' 63.89 ms
geometry=h2o_geometry, basis='sto3g' 77.71 ms
geometry=h2o_geometry, basis='631g' 182.68 ms


[57.631758600473404,
 60.598852299153805,
 63.89132118783891,
 77.71087591536343,
 182.67508065328002]

We compare the relative performance.

In [12]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[1.016172133902563,
 1.65560000377577,
 4.589227348955792,
 5.81479825444936,
 142.88748452916096]

We see again that, as the problem size increases, `qiskit_alt` is increasingly more performant.

## Discussion

The Julia implemenation consists of these packages

* [`QuantumOps.jl`](https://github.ibm.com/IBM-Q-Software/QuantumOps.jl) implementing Fermionic and Pauli operators and calculations using them.

* [`ElectronicStructure.jl`](https://github.ibm.com/IBM-Q-Software/ElectronicStructure.jl) provides an interface to electronic structure packages. Only one is supported, `pyscf`.

* [`QiskitQuantumInfo.jl`](https://github.ibm.com/IBM-Q-Software/QiskitQuantumInfo.jl) provides data structures that mirror Python Qiskit data structures. These are used as intermedidate structures for converting from `QuantumOps` and `ElectronicStructure` to Python Qiskit. In the future these might be used directly for calculations.


The Python interface is a Python package `qiskit_alt`. This could contain a mixture of Julia and Python code. Or all the Julia code might be moved to the Julia packages.

### Implementation

In the examples above, the following happens.

* Julia code calls `pyscf` and stores the results in Julia data structures.

* These data are used to construct a Fermionic operator as a data structure defined in `QuantumOps`.

* The Jordan-Wigner transform, implemented in `QuantumOps` is used to compute a Pauli operator.

* The Pauli operator (as a structure in `QuantumOps`) is converted to a Qiskit-like operator defined in `QiskitQuantumInfo.jl`.

* The operator defined in `QiskitQuantumInfo.jl` is sent to Python and converted to numpy arrays, which are then used to construct native Qiskit types. The conversion to numpy arrays is provided by `pyjulia`.

### Complexity, dynamism

* It is worth noting that operators in `QuantumOps` are *not* highly optimized implementations. In fact, much of the code for the two types of operators is shared, they inherit from a parent class. There are other implementations of Pauli operators in Julia that are much more efficient for instance in [`QuantumClifford.jl`](https://github.com/Krastanov/QuantumClifford.jl).

* [Issue](https://github.ibm.com/IBM-Q-Software/QuantumOps.jl/issues/6) for improving performance of Jordan-Wigner in `QuantumOps`.
    * Precompute one and two-body terms
    * Use @threads


# More demos

Here are some smaller scale demonstrations.

## Converting a matrix to the Pauli basis

Here we convert a matrix representing an operator in the computational basis to the Pauli basis.
In this case, `qiskit_alt` is much more performant than `qiskit.quantum_info`.
This is how it is done in `QuantumOps`.

In [13]:
from qiskit_alt import QuantumOps, PauliSum_to_SparsePauliOp
import numpy as np
m = np.random.rand(2**3, 2**3) # 3-qubit operator
pauli_sum = QuantumOps.PauliSum(m) # This is a wrapped Julia object
PauliSum_to_SparsePauliOp(pauli_sum) # Convert to qiskit.quantum_info.SparsePauliOp

SparsePauliOp(['III', 'XII', 'YII', 'ZII', 'IXI', 'XXI', 'YXI', 'ZXI', 'IYI', 'XYI', 'YYI', 'ZYI', 'IZI', 'XZI', 'YZI', 'ZZI', 'IIX', 'XIX', 'YIX', 'ZIX', 'IXX', 'XXX', 'YXX', 'ZXX', 'IYX', 'XYX', 'YYX', 'ZYX', 'IZX', 'XZX', 'YZX', 'ZZX', 'IIY', 'XIY', 'YIY', 'ZIY', 'IXY', 'XXY', 'YXY', 'ZXY', 'IYY', 'XYY', 'YYY', 'ZYY', 'IZY', 'XZY', 'YZY', 'ZZY', 'IIZ', 'XIZ', 'YIZ', 'ZIZ', 'IXZ', 'XXZ', 'YXZ', 'ZXZ', 'IYZ', 'XYZ', 'YYZ', 'ZYZ', 'IZZ', 'XZZ', 'YZZ', 'ZZZ'],
              coeffs=[ 0.62235558+0.j        ,  0.47977893+0.j        ,  0.        -0.05107663j,
 -0.15073806+0.j        ,  0.44262314+0.j        ,  0.57676216+0.j        ,
  0.        +0.02944513j, -0.02885004+0.j        ,  0.        +0.1170261j ,
  0.        +0.03030807j,  0.05253006+0.j        ,  0.        -0.16851576j,
  0.02910985+0.j        ,  0.07885486+0.j        ,  0.        -0.16002777j,
  0.1483249 +0.j        ,  0.46502834+0.j        ,  0.40536965+0.j        ,
  0.        +0.07793267j,  0.19607602+0.j        ,  0.40226

Note that the `numpy` matrix was automatically converted to a Julia `Matrix`.

We run benchmarks of conversion of matrices to the Pauli basis.

In [14]:
%run ../bench/from_matrix_quantum_info.py

nqubits=2, 1.19 ms
nqubits=3, 4.85 ms
nqubits=4, 23.62 ms
nqubits=5, 135.84 ms
nqubits=6, 995.35 ms
nqubits=7, 10654.28 ms
nqubits=8, 194580.48 ms


In [15]:
%run ../bench/from_matrix_alt.py

nqubits=2, 0.72 ms
nqubits=3, 0.78 ms
nqubits=4, 3.41 ms
nqubits=5, 5.31 ms
nqubits=6, 19.83 ms
nqubits=7, 110.98 ms
nqubits=8, 662.36 ms


Here are the ratios of the times for `qiskit.quantum_info` to those for `qiskit_alt`.

In [16]:
[t_pyqk / t_qk_alt for t_pyqk, t_qk_alt in zip(pyqk_times, qk_alt_times)]

[1.6564846452867465,
 6.195476773470375,
 6.918247447645109,
 25.580937586220454,
 50.1948806920082,
 96.00482290866599,
 293.7697171593044]

Again, the performance gain increases with the problem size.

## Creating a `SparsePauliOp` from a list of strings


Here, we create a `SparsePauliOp` from a list of `n` strings, each with `k` single-Pauli factors, and simplify the result.

First, using `qiskit.quantum_info`

In [17]:
%run ../bench/pauli_from_list_qinfo.py

k=10, n=10, 0.34207114949822426 ms
k=10, n=100, 1.5324915060773492 ms
k=10, n=1000, 13.639073516242206 ms
k=10, n=5000, 67.2700944589451 ms
k=10, n=10000, 137.18474190682173 ms
k=10, n=100000, 1386.441714130342 ms
k=100, n=10, 0.36277950275689363 ms
k=100, n=100, 1.6539825592190027 ms
k=100, n=1000, 14.70456684473902 ms
k=100, n=5000, 72.65388190280646 ms
k=100, n=10000, 149.53304128721356 ms
k=100, n=100000, 1535.1895564235747 ms


Now, using `qiskit_alt`

In [18]:
%run ../bench/pauli_from_list_alt.py

k=10, n=10, 0.6790229585021734 ms
k=10, n=100, 0.798408268019557 ms
k=10, n=1000, 1.865752087906003 ms
k=10, n=5000, 10.455173766240478 ms
k=10, n=10000, 14.146078540943563 ms
k=10, n=100000, 196.40979235991836 ms
k=100, n=10, 0.6737428484484553 ms
k=100, n=100, 0.9089794475585222 ms
k=100, n=1000, 2.944899443536997 ms
k=100, n=5000, 12.500327196903527 ms
k=100, n=10000, 24.897085037082434 ms
k=100, n=100000, 375.00052985269576 ms


The results were written to lists `quantum_info_times` and `qkalt_times`. We compare the performance:

In [19]:
[x / y for x,y in zip(quantum_info_times, qkalt_times)]

[0.5037696372634938,
 1.9194334120295091,
 7.310228194116509,
 6.434144086266526,
 9.697722341196002,
 7.0589235774441725,
 0.5384539570139096,
 1.8196039125653778,
 4.9932322399022135,
 5.812158414605632,
 6.00604613208714,
 4.09383303278695]

We see that the performance improvement in `qiskit_alt` is significant, but does not increase with the number of terms `n`. Further benchmarks show that the time required to convert the strings from Python to Julia takes all the time.

We see this in the following.

In [20]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


Generate `1000` ten-qubit Pauli strings.

In [21]:
%julia using Random: randstring
%julia pauli_strings = [randstring("IXYZ", 10) for _ in 1:1000]
None;

Benchmark converting these to a `QuantumOps.PauliSum`. Note that as the sums are always sorted.

In [22]:
%julia using BenchmarkTools: @btime
%julia @btime QuantumOps.PauliSum($pauli_strings)
None;

  351.098 μs (1012 allocations: 134.17 KiB)


Check that we are actually getting the right result.

In [23]:
%julia pauli_sum = QuantumOps.PauliSum(pauli_strings);
%julia println(length(pauli_sum))
%julia println(pauli_sum[1])

1000
10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Complex{Int64}}:
IIIIXIXXYZ * (1 + 0im)


In [24]:
6.9 * 2.29 / .343  # Ratio of time to construct PauliSum via qiskit_alt to time in pure Julia

46.06705539358601

So the pure Julia code is `46` times faster than the qiskit.quantum_info.
**But, the `qiskit.quantum_info` is also extracting a possible phase !**